## Máster en Data Science - ETL

## ETL Project

Autores: 
- Ramón Morillo Barrera / ramon.morillo@cunef.edu
- José Carlos Monescillo Calzado / jose.monescillo@cunef.edu

## Data: Retail Data Analytics

En este notebook se van a realizar el cálculo de métricas de calidad del dato, para establecer un porcentaje de calidad, el cuál mejoraremos con las posteriores transformaciones. Además, calcularemos nuestras métricas establecidas en la tabla de hechos las cuales nos permitirán detectar patrones de compra y analizar a posteriori las diferentes decisiones de negocio que estarán basadas en los resultados obtenidos mediante el proceso de ETL realizado.

***

# EXTRACCIÓN

En esta fase de extracción se llevará a cabo la importación de los diferentes ficheros como dataframes de pandas, además de un merge para poder calcular las métricas y realizar transformaciones de manera más sencilla posteriormente en la parte de transformación.

In [1]:
# librerías

import pandas as pd
import numpy as np

In [2]:
# Carga de datos

df1 = pd.read_csv('../Data/raw/Features data set.csv')
df2 = pd.read_csv('../Data/raw/sales data-set.csv')
df3 = pd.read_csv('../Data/raw/stores data-set.csv')

In [3]:
# Merge de los datasets
df_merged = df1.merge(df2, on='Store').merge(df3, on='Store')
df_merged.head()

,Store,Date_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_x,Dept,Date_y,Weekly_Sales,IsHoliday_y,Type,Size
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,05/02/2010,24924.50,False,A,151315
1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,12/02/2010,46039.49,True,A,151315
2,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,19/02/2010,41595.55,False,A,151315
3,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,26/02/2010,19403.54,False,A,151315
4,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,05/03/2010,21827.90,False,A,151315


***

# TRANSFORMACIÓN

En esta parte de transformación vamos a realizar las siguientes tareas en el orden establecido:
1. Cálculo de métricas de calidad del dato. Se establecerá la calidad de cada métrica calculada en el informe así como la calidad total de los datos.
2. Imputación de valores faltantes, nulos y transformaciones correspondiendtes según el criterio establecido.
3. Cálculo de métricas de negocio que formarán nuestra tabla de hechos.

## 1. Cáculo de métricas de calidad del dato

### Precisión

Hemos establecido que todas las columnas de tipo 'float' de nuestro dataset teng el mismo número de decimales, en nuestro caso 2 decimales

In [6]:
def count_two_decimals(df):
    float_cols = df.select_dtypes(include=['float64']).columns
    result = {}
    
    for col in float_cols:
        total_values = df[col].dropna().size
        two_decimal_values = df[col].dropna().apply(lambda x: len(str(x).split('.')[-1]) == 2).sum()
        percentage = (two_decimal_values / total_values) * 100
        result[col] = percentage
    
    return result

# Llamar a la función con el dataframe df_merged
two_decimal_percentages = count_two_decimals(df_merged)
print(two_decimal_percentages)

{'Temperature': 89.62670285095979, 'Fuel_Price': 8.984566326763352, 'MarkDown1': 89.63716002890983, 'MarkDown2': 76.32266394123506, 'MarkDown3': 76.58601427946374, 'MarkDown4': 88.11761918581287, 'MarkDown5': 90.19525951931008, 'CPI': 0.2504725572890181, 'Unemployment': 7.224744414826908, 'Weekly_Sales': 82.35263420072586}


### Completitud

Evaluamos el porcentaje de valores no nulos que presentan cada columna en nuestro dataset

In [7]:
def non_null_percentage(df):
    result = {}
    for col in df.columns:
        non_null_count = df[col].notnull().sum()
        total_count = len(df[col])
        percentage = (non_null_count / total_count) * 100
        result[col] = percentage
    return result

# Llamar a la función con el dataframe df_merged
non_null_percentages = non_null_percentage(df_merged)
print(non_null_percentages)

{'Store': 100.0, 'Date_x': 100.0, 'Temperature': 100.0, 'Fuel_Price': 100.0, 'MarkDown1': 49.28840308350236, 'MarkDown2': 36.73842572258019, 'MarkDown3': 44.98602685356961, 'MarkDown4': 44.07547975425196, 'MarkDown5': 49.45054945054945, 'CPI': 92.85714285714286, 'Unemployment': 92.85714285714286, 'IsHoliday_x': 100.0, 'Dept': 100.0, 'Date_y': 100.0, 'Weekly_Sales': 100.0, 'IsHoliday_y': 100.0, 'Type': 100.0, 'Size': 100.0}


### Razonabilidad
Analizamos la razonabilidad a través de los valores mayores o iguales a cero para las columnas numéricas.

Para las columnas de tipo fecha la vamos a analizar observando que los registros estén entre un rango de tiempo coherente y sean anteriores a la fecha actual.

In [4]:
def razonabilidad_general(df):
    razonabilidad_scores = {}

    for col in df.columns:
        
        if pd.api.types.is_numeric_dtype(df[col]):
            
            razonables = (df[col] >= 0)
            razonabilidad_scores[col] = razonables.sum() / len(df) * 100

        
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
    
            min_date = datetime(1990, 1, 1)
            max_date = datetime(2100, 12, 31)
            today = datetime.today()

            razonables = (df[col] >= min_date) & (df[col] <= max_date) & (df[col] <= today)
            razonabilidad_scores[col] = razonables.sum() / len(df) * 100

        elif pd.api.types.is_bool_dtype(df[col]):
            
            razonables = df[col].notnull()
            razonabilidad_scores[col] = razonables.sum() / len(df) * 100
        
        elif col in ['Type']:
            
            validos = ['A', 'B', 'C']
            razonables = df[col].isin(validos)
            razonabilidad_scores[col] = razonables.sum() / len(df) * 100


    return razonabilidad_scores

In [5]:
razonabilidad_scores = razonabilidad_general(df_merged)

# Mostrar resultados
print("Razonabilidad por columna:")
for col, score in razonabilidad_scores.items():
    if score is not None:
        print(f"Columna '{col}': {score:.2f}%")

Razonabilidad por columna:
Columna 'Store': 100.00%
Columna 'Temperature': 99.95%
Columna 'Fuel_Price': 100.00%
Columna 'MarkDown1': 49.25%
Columna 'MarkDown2': 36.42%
Columna 'MarkDown3': 44.83%
Columna 'MarkDown4': 44.08%
Columna 'MarkDown5': 49.43%
Columna 'CPI': 92.86%
Columna 'Unemployment': 92.86%
Columna 'IsHoliday_x': 100.00%
Columna 'Dept': 100.00%
Columna 'Weekly_Sales': 99.70%
Columna 'IsHoliday_y': 100.00%
Columna 'Type': 100.00%
Columna 'Size': 100.00%


### Estructura

In [6]:
def estructura(df):
    estructura_scores = {}

    for col in df.columns:
        # Si la columna es numérica
        if pd.api.types.is_numeric_dtype(df[col]):
            
            valid_structure = df[col].notnull() & df[col].map(type).isin([int, float])
            estructura_scores[col] = valid_structure.sum() / len(df) * 100

        # Si la columna es categórica
        elif pd.api.types.is_object_dtype(df[col]):
            
            valid_structure = df[col].notnull() & df[col].map(type).eq(str)
            estructura_scores[col] = valid_structure.sum() / len(df) * 100

        # Si la columna es de tipo fecha (Date_x o Date_y)
        elif col in ['Date_x', 'Date_y']:
            # Regla: Formato XX/XX/XXXX
            # Convrtimos a string
            
            valid_structure = df[col].astype(str).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False)
            estructura_scores[col] = valid_structure.sum() / len(df) * 100

            # Volver a convertir a datetime
            df[col] = pd.to_datetime(df[col], format='%d/%m/%Y')
        
        # Si la columna es de tipo booleano
        elif pd.api.types.is_bool_dtype(df[col]):

            valid_structure = df[col].isin([True, False, 1, 0]) & df[col].notnull()
            estructura_scores[col] = valid_structure.sum() / len(df) * 100


    return estructura_scores

In [7]:
estructura_scores = estructura(df_merged)

print("\nEstructura por columna:")
for col, score in estructura_scores.items():
        
        print(f"Columna '{col}': {score:.2f}%")


Estructura por columna:
Columna 'Store': 100.00%
Columna 'Date_x': 100.00%
Columna 'Temperature': 100.00%
Columna 'Fuel_Price': 100.00%
Columna 'MarkDown1': 49.29%
Columna 'MarkDown2': 36.74%
Columna 'MarkDown3': 44.99%
Columna 'MarkDown4': 44.08%
Columna 'MarkDown5': 49.45%
Columna 'CPI': 92.86%
Columna 'Unemployment': 92.86%
Columna 'IsHoliday_x': 0.00%
Columna 'Dept': 100.00%
Columna 'Date_y': 100.00%
Columna 'Weekly_Sales': 100.00%
Columna 'IsHoliday_y': 0.00%
Columna 'Type': 100.00%
Columna 'Size': 100.00%


### Identificabilidad

Obtenemos la identificabilidad con la proporción de valores unicos y posteriormente su porcentaje.

In [10]:
def identificabilidad_unicos(df):
    identificabilidad_scores = {}
    for col in df.columns:
        unique_count = df[col].nunique()
        total_count = len(df[col])
        identificabilidad_score = (unique_count / total_count) * 100
        identificabilidad_scores[col] = identificabilidad_score
    return identificabilidad_scores


identificabilidad_scores_unicos = identificabilidad_unicos(df_merged)
print("\n### Identificabilidad por columna")
for col, score in identificabilidad_scores_unicos.items():
    print(f"Identificabilidad de '{col}': {score:.2f}%")


### Identificabilidad por columna
Identificabilidad de 'Store': 0.00%
Identificabilidad de 'Date_x': 0.00%
Identificabilidad de 'Temperature': 0.01%
Identificabilidad de 'Fuel_Price': 0.00%
Identificabilidad de 'MarkDown1': 0.01%
Identificabilidad de 'MarkDown2': 0.00%
Identificabilidad de 'MarkDown3': 0.00%
Identificabilidad de 'MarkDown4': 0.00%
Identificabilidad de 'MarkDown5': 0.01%
Identificabilidad de 'CPI': 0.00%
Identificabilidad de 'Unemployment': 0.00%
Identificabilidad de 'IsHoliday_x': 0.00%
Identificabilidad de 'Dept': 0.00%
Identificabilidad de 'Date_y': 0.00%
Identificabilidad de 'Weekly_Sales': 0.47%
Identificabilidad de 'IsHoliday_y': 0.00%
Identificabilidad de 'Type': 0.00%
Identificabilidad de 'Size': 0.00%


## 2. Transformación de los datos

### Imputación de valores faltantes

En primer lugar, hemos decidido imputar los valores de las columnas 'Markdown 1-5' por el valor 0, ya que no tenemos información acerca de los datos. Son datos anónimos que explican rebajas promocionales, por tanto, intuímos que un dato faltante es un día en el no hubo ningún tipo de promoción, es por ello que se imputarán por 0.

A su vez, imputaremos los valores faltantes de la columna CPI (Índice de precios al consumidor) y de la columna Unemployment (Desempleo) por su valor mediano, esto es ya que, no hay muchos valores nulos y estadísticamente esta imputación no afectará mucho ya que son valores que no suelen tener cambios muy acentuados de periodo en periodo. 

In [4]:
# Imputación de valores nulos
df_merged[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']] = df_merged[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].fillna(0)
df_merged['CPI'] = df_merged['CPI'].fillna(df_merged['CPI'].median())
df_merged['Unemployment'] = df_merged['Unemployment'].fillna(df_merged['Unemployment'].median())

# Verificar que los valores nulos han sido imputados
print(df_merged[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']].isnull().sum())

MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
dtype: int64


### Establecer 2 decimales a variables 'float'

Para mejorar la precisión, como hemos expuesto anteriormente, vamos a establecer que todas las columnas float presenten 2 decimales.

In [5]:
df_merged = df_merged.round(2)
df_merged.head()

,Store,Date_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_x,Dept,Date_y,Weekly_Sales,IsHoliday_y,Type,Size
0,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1,8.11,False,1,05/02/2010,24924.50,False,A,151315
1,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1,8.11,False,1,12/02/2010,46039.49,True,A,151315
2,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1,8.11,False,1,19/02/2010,41595.55,False,A,151315
3,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1,8.11,False,1,26/02/2010,19403.54,False,A,151315
4,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1,8.11,False,1,05/03/2010,21827.90,False,A,151315


# 3. Cálculo de métricas de negocio

En esta parte de la transformación se calcularán las siguientes métricas que se alojarán en nuestra tabla de hechos:
- Ventas semanales totales por departamento
- Total de ventas por tienda en días festivos
- Tamaño total de tiendas según su tipo
- Tasa de variación media del ‘CPI’ a lo largo de los 3 años
- Tasa de variación media del ‘Fuel_price’ a lo largo de los 3 años

### Ventas semanales totales por departamento

In [6]:
# Calcular la suma de Weekly_Sales para cada departamento
suma_ventas_por_depto = df2.groupby('Dept')['Weekly_Sales'].sum().reset_index()
suma_ventas_por_depto.rename(columns={'Weekly_Sales': 'suma_ventas'}, inplace=True)

# Añadir separadores de miles y redondear a 2 decimales
suma_ventas_por_depto['suma_ventas'] = suma_ventas_por_depto['suma_ventas'].apply(lambda x: f"{x:,.2f}")

suma_ventas_por_depto

,Dept,suma_ventas
0,1,"123,638,776.54"
1,2,"280,611,174.43"
2,3,"75,892,449.95"
3,4,"167,146,745.58"
4,5,"135,607,358.57"
...,...,...
76,95,"449,320,162.52"
77,96,"73,833,916.16"
78,97,"89,496,511.90"
79,98,"39,828,919.37"


### Total de ventas por tienda en días festivos

In [7]:
# Filtrar el dataframe para incluir solo las filas donde IsHoliday es True
ventas_dias_festivos = df2[df2['IsHoliday'] == True]

# Calcular la suma de Weekly_Sales para cada tienda
total_ventas_por_tienda_festivos = ventas_dias_festivos.groupby('Store')['Weekly_Sales'].sum().reset_index()
total_ventas_por_tienda_festivos.rename(columns={'Weekly_Sales': 'total_ventas'}, inplace=True)

# Añadir separadores de miles y redondear a 2 decimales
total_ventas_por_tienda_festivos['total_ventas'] = total_ventas_por_tienda_festivos['total_ventas'].apply(lambda x: f"{x:,.2f}")

total_ventas_por_tienda_festivos

,Store,total_ventas
0,1,"16,657,476.56"
1,2,"20,792,669.00"
2,3,"4,378,110.50"
3,4,"22,431,026.24"
4,5,"3,595,016.07"
5,6,"16,809,079.27"
6,7,"6,724,002.65"
7,8,"9,753,308.60"
8,9,"5,889,508.21"
9,10,"21,137,559.49"


### Tamaño total de tiendas según su tipo

In [8]:
# Agrupar por el tipo de tienda y sumar los tamaños
tamaño_total_por_tipo = df3.groupby('Type')['Size'].sum().reset_index()
tamaño_total_por_tipo.rename(columns={'Size': 'tamaño_total'}, inplace=True)

tamaño_total_por_tipo

,Type,tamaño_total
0,A,3899450
1,B,1720242
2,C,243250


### Tasa de variación media del ‘CPI’ a lo largo de los 3 años

In [9]:
# Convertir la columna Date a formato de fecha
df1['Date'] = pd.to_datetime(df1['Date'], format='%d/%m/%Y')

# Extraer el año de la columna Date
df1['Year'] = df1['Date'].dt.year

# Calcular la tasa de variación anual del CPI
df_cpi = df1.groupby('Year')['CPI'].mean().pct_change() * 100

# Crear un dataframe con el año y la tasa de variación del CPI
df_tasa_variacion_cpi = df_cpi.reset_index()
df_tasa_variacion_cpi.columns = ['Year', 'Tasa_Variacion_CPI']

# Redondear a 2 decimales
df_tasa_variacion_cpi['Tasa_Variacion_CPI'] = df_tasa_variacion_cpi['Tasa_Variacion_CPI'].round(2)

df_tasa_variacion_cpi

,Year,Tasa_Variacion_CPI
0,2010,NaN
1,2011,2.05
2,2012,2.43
3,2013,1.08


### Tasa de variación media del ‘Fuel_price’ a lo largo de los 3 años

In [10]:
# Calcular la tasa de variación anual del Fuel_Price
df_fuel_price = df1.groupby('Year')['Fuel_Price'].mean().pct_change() * 100

# Crear un dataframe con el año y la tasa de variación del Fuel_Price
df_tasa_variacion_fuel_price = df_fuel_price.reset_index()
df_tasa_variacion_fuel_price.columns = ['Year', 'Tasa_Variacion_Fuel_Price']

# Redondear a 2 decimales
df_tasa_variacion_fuel_price['Tasa_Variacion_Fuel_Price'] = df_tasa_variacion_fuel_price['Tasa_Variacion_Fuel_Price'].round(2)

df_tasa_variacion_fuel_price

,Year,Tasa_Variacion_Fuel_Price
0,2010,NaN
1,2011,26.14
2,2012,3.09
3,2013,-1.80


***

# CARGA

En este apartado final se crearán los diversos dataframes que se ajusten a nuestro modelo relacional en estrella expuesto en el informe, con esto conseguimos tener un modelo relacional que se une por una columna común. Una tabla de hechos que facilita la toma de decisiones empresariales en función de sus métricas calculadas y la posibilidad de combinar tablas para cálculo de nuevas métricas debido a la columna común Store (id de cada tienda).

### Features Data

In [11]:
# Crear un nuevo dataframe
features_data = df_merged[['Store', 'Date_x', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI']]

# Renombrar la columna 'Date_x' a 'Date'
features_data.rename(columns={'Date_x': 'Date'}, inplace=True)

features_data.head()

C:\Users\RAMON\AppData\Local\Temp\ipykernel_10868\3460687807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_data.rename(columns={'Date_x': 'Date'}, inplace=True)


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI
0,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1
1,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1
2,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1
3,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1
4,1,05/02/2010,42.31,2.57,0.0,0.0,0.0,0.0,0.0,211.1


### Sales Data

In [12]:
# Crear el nuevo dataframe
sales_data = df_merged[['Store', 'Dept', 'Date_y', 'Weekly_Sales', 'IsHoliday_y']]

# Renombrar las columnas 'Date_y' a 'Date' y 'IsHoliday_y' a 'IsHoliday'
sales_data.rename(columns={'Date_y': 'Date', 'IsHoliday_y': 'IsHoliday'}, inplace=True)


sales_data.head()

C:\Users\RAMON\AppData\Local\Temp\ipykernel_10868\2566436803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data.rename(columns={'Date_y': 'Date', 'IsHoliday_y': 'IsHoliday'}, inplace=True)


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


### Stores Data

In [21]:
# Crear el nuevo dataframe
stores_data = df_merged[['Store', 'Type', 'Size']].drop_duplicates()

# Restablecer el índice
stores_data.reset_index(drop=True, inplace=True)

stores_data.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


### Tabla de Hechos

In [25]:
# Crear el dataframe tabla_de_hechos
tabla_de_hechos = pd.DataFrame({
    'Store': total_ventas_por_tienda_festivos['Store'].astype(int),
    'Date': df1['Date'],
    'suma_ventas': suma_ventas_por_depto['suma_ventas'],
    'total_ventas': total_ventas_por_tienda_festivos['total_ventas'],
    'tamaño_total': tamaño_total_por_tipo['tamaño_total'],
    'Tasa_Variacion_CPI': df_tasa_variacion_cpi['Tasa_Variacion_CPI'],
    'Tasa_Variacion_Fuel_Price': df_tasa_variacion_fuel_price['Tasa_Variacion_Fuel_Price']
})

# Convertir la columna Store a enteros
tabla_de_hechos['Store'] = tabla_de_hechos['Store'].fillna(0).astype(int)

# Mostrar el dataframe resultante
tabla_de_hechos.head()

,Store,Date,suma_ventas,total_ventas,tamaño_total,Tasa_Variacion_CPI,Tasa_Variacion_Fuel_Price
0,1,2010-02-05,"123,638,776.54","16,657,476.56",3899450.0,NaN,NaN
1,2,2010-02-12,"280,611,174.43","20,792,669.00",1720242.0,2.05,26.14
2,3,2010-02-19,"75,892,449.95","4,378,110.50",243250.0,2.43,3.09
3,4,2010-02-26,"167,146,745.58","22,431,026.24",NaN,1.08,-1.80
4,5,2010-03-05,"135,607,358.57","3,595,016.07",NaN,NaN,NaN


### Dim_tiempo

In [27]:
# Crear el dataframe dim_tiempo
dim_tiempo = df1[['Date']].drop_duplicates().reset_index(drop=True)
dim_tiempo


,Date
0,2010-02-05
1,2010-02-12
2,2010-02-19
3,2010-02-26
4,2010-03-05
...,...
177,2013-06-28
178,2013-07-05
179,2013-07-12
180,2013-07-19


Con este procedimiento codificado con Python, redactado y comentado, habríamos acabado nuestro proceso de ETL. Toda la información y la orientación del proyecto en cuanto a negocio se encuentra en el informe en formato PDF, además de las conclusiones finales.